In [1]:
import pandas as pd
import dvc.api as DvcApi
import cohere

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [ ]:
# Paste your API key here. Remember to not share publicly
api_key = 'qPKnuy0sl0o4eg2bQ9nOe6qVmzTZoYcZT28DXvoN'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

# Named Entity Recognition (NER)  with Generative Model of Cohere

In this notebook we use Cohere's generative models to extract entities from a job description. We make use of sturctured generation based on providing multiple examples in the prompt.

The data are job descriptions (together named entities) and relationships between entities in json format. In this preliminary work we will not consider the relationships as NER alone is not enough to achieve this.

- Dataset 1: For development and training
- Dataset 2: For testing and final reporting


In [ ]:
# Paste your API key here. Remember to not share publicly
api_key = 'qPKnuy0sl0o4eg2bQ9nOe6qVmzTZoYcZT28DXvoN'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

## Preparing examples for the prompt

In our prompt, we'll present the model with examples of job descriptions and the type of output we're after.

In [ ]:
path = "data/relations_dev.json"
repo = "../"
version = "v2"

data_url = DvcApi.get_url(path = path, repo = repo, rev = version) #could be tag or git commit
data_dev = pd.read_csv(data_url)